# 230623_causal_tracing

Hooking 방식 좀 더 안전하게 변경  
tokenisation 할 때 max_sequence 부여

# Import libraries

In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer, set_seed
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from datetime import date
import sys
import os


/rds/general/user/jj1122/home/anaconda3/envs/m2d2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# # module params

In [3]:
import json
with open("./data/dict_n_module_params_230623.json", "r") as json_file:
    dict_temp = json.load(json_file)

In [27]:
def parse_code(row):
    list_code = row.code.split(".")
    is_in_layer = row["code"].startswith("transformer.h")
    if is_in_layer:        
        row["l"] = int(list_code[2])
        row["m"] = list_code[3]
    else:
        row["m"] = list_code[1]
        if row["m"] == "ln_f": 
            row["l"] = int(99)
        elif row["m"] in ["wte", "wpe"]: 
            row['l'] = int(-1)
    row["w_or_b"] = list_code[-1]
    row["is_in_layer"] = is_in_layer
    return row

df_temp = pd.DataFrame.from_dict(dict_temp, orient="index").reset_index()
df_temp.columns = ['code', 'n_params']
df_temp = df_temp.apply(lambda row: parse_code(row), axis=1)
df_temp

,code,is_in_layer,l,m,n_params,w_or_b
0,transformer.wte.weight,False,-1,wte,38597376,weight
1,transformer.wpe.weight,False,-1,wpe,786432,weight
2,transformer.h.0.ln_1.weight,True,0,ln_1,768,weight
3,transformer.h.0.ln_1.bias,True,0,ln_1,768,bias
4,transformer.h.0.attn.c_attn.weight,True,0,attn,1769472,weight
...,...,...,...,...,...,...
143,transformer.h.11.mlp.c_fc.bias,True,11,mlp,3072,bias
144,transformer.h.11.mlp.c_proj.weight,True,11,mlp,2359296,weight
145,transformer.h.11.mlp.c_proj.bias,True,11,mlp,768,bias
146,transformer.ln_f.weight,False,99,ln_f,768,weight


In [33]:
df_temp.loc[df_temp.w_or_b == "weight", ["m", "n_params", "is_in_layer"]]\
    .drop_duplicates()\
    .groupby(["is_in_layer", "m"], as_index=False)[["n_params"]].sum().drop_duplicates()

,is_in_layer,m,n_params
0,False,ln_f,768
1,False,wpe,786432
2,False,wte,38597376
3,True,attn,2359296
4,True,ln_1,768
5,True,ln_2,768
6,True,mlp,2359296


In [ ]:
def parse_code(row):
    list_code = row.code.split(".")
    row['l'] = int(list_code[2])
    row['m'] = list_code[3]
#     row['t'] = int(list_code[4])
    return row

list_df_ide_temp = []
for i, d in enumerate(list_results_temp):
    df_t = pd.DataFrame.from_dict(d['restored_loss'], orient='index')
    TE = (corrupted_loss - clean_loss) / clean_loss
    IDE = {}
    for m_id in list_trace_module_ids:
        IDE[m_id] = (restored_loss[m_id] - clean_loss) / clean_loss
    
    df_ide = pd.DataFrame.from_dict(IDE, orient='index').reset_index()
    df_ide.columns = ['code', 'ide']
    df_ide = df_ide.apply(lambda row: parse_code(row), axis=1)
    df_ide["seq"] = i
    list_df_ide_temp.append(df_ide)
    break